In [ ]:
%load_ext autoreload
%autoreload 


In [ ]:
import timeit
import copy
import concurrent.futures 
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import batoid

from ghosts import plotter
from ghosts import simulator
from ghosts import tweak_optics
from ghosts import beam
from ghosts import geom


from ghosts.analysis import compute_ghost_separations, reduce_ghosts
from ghosts.analysis import match_ghosts, compute_reduced_distance, compute_2d_reduced_distance, find_nearest_ghost
from ghosts.analysis import make_data_frame

from ghosts.beam_configs import BEAM_CONFIG_0, BEAM_CONFIG_1, BEAM_CONFIG_2, FAST_BEAM_CONFIG_1
from ghosts.geom_config import GEOM_CONFIG_0


In [ ]:
from ghosts.constants import *
print(f'LSST Camera has {LSST_CAMERA_PIXEL_DENSITY_MM2:.0f} pixels per mm^2')

In [ ]:
# CCOB like geometry, i.e. lenses + filters
telescope = batoid.Optic.fromYaml("../data/LSST_CCOB_r.yaml")

In [ ]:
# test list of optics
assert tweak_optics.get_list_of_optics(telescope)==['L1', 'L2', 'Filter', 'L3', 'Detector'], 'Not a CCOB optical setup'

# Make refractive interfaces partially reflective
# Call on current telescope
tweak_optics.make_optics_reflective(telescope)


In [ ]:
# Run simulation with standard beam config
traceFull, rForward, rReverse, rays = simulator.run_simulation(telescope, beam_config=BEAM_CONFIG_1)


In [ ]:
# Analyze
spots_data, _spots = reduce_ghosts(rForward)
data_frame = make_data_frame(spots_data, BEAM_CONFIG_1['beam_id'], GEOM_CONFIG_0['geom_id'])


In [ ]:
data_frame.head()

In [ ]:
# Test building several beams
x_scan = [0., 0.05, 0.1, 0.15, 0.2]
t_scan = [0, 5, 10, 15]

In [ ]:
beam_scan_x = beam.build_translation_set(BEAM_CONFIG_0, 'x', x_scan, base_id=10000)
beam_scan_t = beam.build_rotation_set(BEAM_CONFIG_1, 'theta', t_scan, base_id=20000)

In [ ]:
beam_scan_x_panda = beam.concat_dicts(beam_scan_x)
beam_scan_x_panda.head()

In [ ]:
beam_scan_t_panda = beam.concat_dicts(beam_scan_t)
beam_scan_t_panda.head()

In [ ]:
# test make fast beams on the data frame for all beam configurations
beam_scan_t_panda['n_photons']=10
beam_scan_t_panda.head()